The telecommunications (telecom) sector in India is rapidly changing, with more and more telecom businesses being created and many customers deciding to switch between providers. "Churn" refers to the process where customers or subscribers stop using a company's services or products. Understanding the factors that influence keeping a customer as a client in predicting churn is crucial for telecom companies to enhance their service quality and customer satisfaction. As the data scientist on this project, you aim to explore the intricate dynamics of customer behavior and demographics in the Indian telecom sector in predicting customer churn, utilizing two comprehensive datasets from four major telecom partners: Airtel, Reliance Jio, Vodafone, and BSNL:

- `telecom_demographics.csv` contains information related to Indian customer demographics:

| Variable             | Description                                      |
|----------------------|--------------------------------------------------|
| `customer_id `         | Unique identifier for each customer.             |
| `telecom_partner `     | The telecom partner associated with the customer.|
| `gender `              | The gender of the customer.                      |
| `age `                 | The age of the customer.                         |
| `state`                | The Indian state in which the customer is located.|
| `city`                 | The city in which the customer is located.       |
| `pincode`              | The pincode of the customer's location.          |
| `registration_event` | When the customer registered with the telecom partner.|
| `num_dependents`      | The number of dependents (e.g., children) the customer has.|
| `estimated_salary`     | The customer's estimated salary.                 |

- `telecom_usage` contains information about the usage patterns of Indian customers:

| Variable   | Description                                                  |
|------------|--------------------------------------------------------------|
| `customer_id` | Unique identifier for each customer.                         |
| `calls_made` | The number of calls made by the customer.                    |
| `sms_sent`   | The number of SMS messages sent by the customer.             |
| `data_used`  | The amount of data used by the customer.                     |
| `churn`    | Binary variable indicating whether the customer has churned or not (1 = churned, 0 = not churned).|


In [116]:
# Import libraries and methods/functions
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix


In [117]:
from sklearn.metrics import accuracy_score

In [ ]:
#loading the datasets
demographics = pd.read_csv('telecom_demographics.csv')
usage = pd.read_csv('telecom_usage.csv')

In [118]:
demographics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6500 entries, 0 to 6499
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         6500 non-null   int64 
 1   telecom_partner     6500 non-null   object
 2   gender              6500 non-null   object
 3   age                 6500 non-null   int64 
 4   state               6500 non-null   object
 5   city                6500 non-null   object
 6   pincode             6500 non-null   int64 
 7   registration_event  6500 non-null   object
 8   num_dependents      6500 non-null   int64 
 9   estimated_salary    6500 non-null   int64 
dtypes: int64(5), object(5)
memory usage: 507.9+ KB


In [119]:
usage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6500 entries, 0 to 6499
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   customer_id  6500 non-null   int64
 1   calls_made   6500 non-null   int64
 2   sms_sent     6500 non-null   int64
 3   data_used    6500 non-null   int64
 4   churn        6500 non-null   int64
dtypes: int64(5)
memory usage: 254.0 KB


In [120]:
#merging the two datasets into churn_df
churn_df = demographics.merge(usage, on='customer_id')

In [121]:
churn_df.head()

,customer_id,telecom_partner,gender,age,state,city,pincode,registration_event,num_dependents,estimated_salary,calls_made,sms_sent,data_used,churn
0,15169,Airtel,F,26,Himachal Pradesh,Delhi,667173,2020-03-16,4,85979,75,21,4532,1
1,149207,Airtel,F,74,Uttarakhand,Hyderabad,313997,2022-01-16,0,69445,35,38,723,1
2,148119,Airtel,F,54,Jharkhand,Chennai,549925,2022-01-11,2,75949,70,47,4688,1
3,187288,Reliance Jio,M,29,Bihar,Hyderabad,230636,2022-07-26,3,34272,95,32,10241,1
4,14016,Vodafone,M,45,Nagaland,Bangalore,188036,2020-03-11,4,34157,66,23,5246,1


In [122]:
#calculating the churn rate
churn_rate = (churn_df.churn.value_counts(normalize=True).loc[1])*100
print(f'The Churn rate is {round(churn_rate,2)}%')

The Churn rate is 20.05%


In [123]:
#getting the target series
target = churn_df.churn

In [124]:
#droping customer id and churn beacause churn is the target and the id is not a relevant feature
churn_df.drop(columns = ['customer_id','churn'], inplace = True)

In [125]:
#identifying the categorical variables in the Dataframe
# All non numeric columns will be considered categorical
cat_columns = ['telecom_partner','gender','state','city','registration_event']
features = pd.get_dummies(data = churn_df,columns = cat_columns)


In [126]:
#THe next step is to scale the features
#intialize the scaler
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)


In [127]:
#spliting the data into train and test samples
X_train,X_test, y_train,y_test = train_test_split(features_scaled, target, test_size = 0.2,random_state = 42)

In [128]:
#training a logistic regression model
#initializing the model
log_reg = LogisticRegression(random_state =42)
#fitting the data
log_reg.fit(X_train,y_train)

LogisticRegression(random_state=42)

In [129]:
#testing the model
logreg_pred = log_reg.predict(X_test)

In [130]:
#assessing the Logistic regression model
classification = classification_report(y_test,logreg_pred)
print(classification)

              precision    recall  f1-score   support

           0       0.79      0.90      0.84      1027
           1       0.21      0.10      0.14       273

    accuracy                           0.73      1300
   macro avg       0.50      0.50      0.49      1300
weighted avg       0.67      0.73      0.69      1300



In [131]:
#training a random forest model
#instantiating the model
forest = RandomForestClassifier(random_state = 42)

#training the model
forest.fit(X_train,y_train)

#testing the model
rf_pred = forest.predict(X_test)

#assessing the model
forest_classification = classification_report(y_test,rf_pred)
print(forest_classification)


              precision    recall  f1-score   support

           0       0.79      1.00      0.88      1027
           1       0.00      0.00      0.00       273

    accuracy                           0.79      1300
   macro avg       0.39      0.50      0.44      1300
weighted avg       0.62      0.79      0.70      1300



In [132]:
higher_accuracy = 'RandomForest'